<a href="https://colab.research.google.com/github/nasrinrastgoo/dataMining_testMininng/blob/main/text_mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import nltk
from bs4 import BeautifulSoup
import string
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from sklearn.model_selection import train_test_split
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
#load dataset
sms = pd.read_csv('SmsCollection-2.csv', delimiter = ';', on_bad_lines='skip')
sms

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [3]:
sms.shape

(5572, 2)

Data preprocessing

In [4]:
# remove punctuation
def remove_punctuation(text):
  text_str=str(text)
  no_punct = "".join([c for c in text_str if c not in string.punctuation])
  return no_punct

In [5]:
sms['text'] = sms['text'].apply(lambda x: remove_punctuation(x))
sms.head()

,label,text
0,ham,Go until jurong point crazy Available only in ...
1,ham,Ok lar Joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor U c already then say
4,ham,Nah I dont think he goes to usf he lives aroun...


first change the text list to a list of words, not an array of characters, then we can remove stop words. 

In [6]:
# tokenize
sms['text'] = sms['text'].apply(lambda x: word_tokenize(x))
sms.head

<bound method NDFrame.head of      label                                               text
0      ham  [Go, until, jurong, point, crazy, Available, o...
1      ham                     [Ok, lar, Joking, wif, u, oni]
2     spam  [Free, entry, in, 2, a, wkly, comp, to, win, F...
3      ham  [U, dun, say, so, early, hor, U, c, already, t...
4      ham  [Nah, I, dont, think, he, goes, to, usf, he, l...
...    ...                                                ...
5567  spam  [This, is, the, 2nd, time, we, have, tried, 2,...
5568   ham       [Will, ü, b, going, to, esplanade, fr, home]
5569   ham  [Pity, was, in, mood, for, that, Soany, other,...
5570   ham  [The, guy, did, some, bitching, but, I, acted,...
5571   ham                   [Rofl, Its, true, to, its, name]

[5572 rows x 2 columns]>

In [7]:
# remove stop words
def remove_stopwords(text):
  words = [w for w in text if w not in stopwords.words('english')]
  return words

In [8]:
sms['text'] = sms['text'].apply(lambda x: remove_stopwords(x))
sms.head(10)

,label,text
0,ham,"[Go, jurong, point, crazy, Available, bugis, n..."
1,ham,"[Ok, lar, Joking, wif, u, oni]"
2,spam,"[Free, entry, 2, wkly, comp, win, FA, Cup, fin..."
3,ham,"[U, dun, say, early, hor, U, c, already, say]"
4,ham,"[Nah, I, dont, think, goes, usf, lives, around..."
5,spam,"[FreeMsg, Hey, darling, 3, weeks, word, back, ..."
6,ham,"[Even, brother, like, speak, They, treat, like..."
7,ham,"[As, per, request, Melle, Melle, Oru, Minnamin..."
8,spam,"[WINNER, As, valued, network, customer, select..."
9,spam,"[Had, mobile, 11, months, U, R, entitled, Upda..."


In [12]:
#lemmatization
lemmatizer = WordNetLemmatizer()

def word_lemmatizer(text):
  lem_text = [lemmatizer.lemmatize(i) for i in text]
  return lem_text

sms['text'] = sms['text'].apply(lambda x: word_lemmatizer(x))

Features

In [13]:
"CountVectorizer"
count_vect = CountVectorizer()
sms_counts = count_vect.fit_transform(sms.text.apply(lambda x: ' '.join(x)))


In [14]:
"tf-idf transformer"
tfidf_transformer = TfidfTransformer()
sms_tfidf = tfidf_transformer.fit_transform(sms_counts)
sms_tfidf.shape

(5572, 8836)


**model preparation**

start with train, test, split; then vectorize the text (transform it into numerical data)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(sms_tfidf, sms['label'], test_size= 0.25, random_state= 2)


In [18]:
#Naive Bayes model
clf = MultinomialNB().fit(X_train, y_train)

In [19]:
predicted = clf.predict(X_test)
np.mean(predicted == y_test)

0.95908111988514

In [20]:
# SVM Model
svm = SGDClassifier().fit(X_train, y_train)

In [21]:
predicted_svm = svm.predict(X_test)
np.mean(predicted_svm == y_test)

0.9741564967695621

In [22]:
#decision tree classification
from sklearn import tree

dtc = tree.DecisionTreeClassifier().fit(X_train, y_train)

In [23]:
predicted = dtc.predict(X_test)
np.mean(predicted == y_test)

0.955491744436468